# Location Tracking using Trilateration
Logan Gall, gall0487
13 Dec. 2023

This file performs the true location tracking using wifi beacons. It takes input of known access point locations, then scans current wifi beacons for signal strength, and finally uses least squares to trilaterate current position.

In [ ]:
import csv
import time
from pywifi import PyWiFi, const
from scipy.optimize import least_squares

In [1]:
# Function to perform trilateration
def trilateration(nodes, distances):
    # Nested function defining the equations for trilateration
    def equations(point):
        x, y, z = point
        # Compute the squared differences between measured and calculated distances
        return [((x - nx)**2 + (y - ny)**2 + (z - nz)**2 - d**2) for [nx, ny, nz], d in zip(nodes, distances)]
    # Starting with an initial guess of [0, 0, 0]
    initial_guess = [0, 0, 0]
    # Using least squares to solve the equations
    result = least_squares(equations, initial_guess)
    return result.x

In [2]:
#Read in file with access point locations
def read_estimated_positions(filename="estimated_positions.csv"):
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        positions = {row['BSSID']: [float(row['X Position']), float(row['Y Position']), float(row['Z Position'])] for row in reader}
    return positions

In [3]:
def scan_wifi_for_trilateration():
    wifi = PyWiFi()
    iface = wifi.interfaces()[0]

    iface.scan()
    time.sleep(3)  # Wait for the scan to complete

    results = iface.scan_results()
    scanned_networks = {}

    for network in results:
        scanned_networks[network.bssid] = abs(int(network.signal))  # Signal strength as positive value

    return scanned_networks

In [4]:
def match_networks_with_positions(scanned_networks, estimated_positions):
    nodes = []
    distances = []

    for bssid, signal_strength in scanned_networks.items():
        if bssid in estimated_positions:
            nodes.append(estimated_positions[bssid])
            distances.append(signal_strength)  # Directly use signal strength as proxy for distance

    return nodes, distances

In [5]:
# Other functions like read_estimated_positions, scan_wifi_for_trilateration,
# match_networks_with_positions, trilateration are assumed to be defined above this part.

# The main function of the script
def main():
    # Read previously estimated positions of WiFi access points
    estimated_positions = read_estimated_positions()
    # List to store all estimated current positions during this session
    all_estimated_current_positions = []

    # Infinite loop to continuously perform WiFi scanning and position estimation
    while True:
        # Prompt the user to initiate a WiFi scan or quit the program
        user_input = input("Press Enter to scan for WiFi and estimate position (type 'quit' and press Enter to exit)...")
        if user_input.lower() == 'quit':
            # Exit the loop if the user types 'quit'
            break

        # Scan for WiFi networks for trilateration purposes
        scanned_networks = scan_wifi_for_trilateration()
        # Match the scanned networks with known positions to get nodes and distances
        nodes, distances = match_networks_with_positions(scanned_networks, estimated_positions)
        
        # Check if there is enough data (nodes and distances) for trilateration
        if nodes and distances:
            # Estimate the current position using trilateration
            current_position = trilateration(nodes, distances)
            print(f"Estimated current position: {current_position}")
            # Add the estimated position to the list
            all_estimated_current_positions.append(current_position)
        else:
            print("Not enough data for trilateration.")

    # Saving the collected position estimates to a CSV file
    with open('current_positions.csv', 'w', newline='') as csvfile:
        fieldnames = ['X Position', 'Y Position', 'Z Position']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        # Writing each estimated position to the CSV file
        for position in all_estimated_current_positions:
            writer.writerow({'X Position': position[0], 'Y Position': position[1], 'Z Position': position[2]})

    # Print a message confirming the saving of position estimates
    print("Current position estimates saved to 'current_positions.csv'.")

Press Enter to scan for WiFi and estimate position (type 'quit' and press Enter to exit)...
Estimated current position: [  0.          63.78036041 -29.03402782]
Press Enter to scan for WiFi and estimate position (type 'quit' and press Enter to exit)...
Estimated current position: [  0.          64.96202839 -35.19963263]
Press Enter to scan for WiFi and estimate position (type 'quit' and press Enter to exit)...
Estimated current position: [  0.          64.40781239 -33.39764969]
Press Enter to scan for WiFi and estimate position (type 'quit' and press Enter to exit)...quit


PermissionError: [Errno 13] Permission denied: 'current_positions.csv'

In [ ]:
main()